# Preparing the word-entailment bake-off dataset

In [1]:
from collections import defaultdict
import json
import pandas as pd
import random
from sklearn.model_selection import train_test_split

From https://github.com/sleepinyourhat/vector-entailment/releases/tag/W15-R2:

In [2]:
wordentail_filename = 'shuffled_word_relations.tsv'

In [3]:
df = pd.read_csv(
    wordentail_filename, delimiter="\t", 
    names=['relation', 'left', 'right'])

In [4]:
df.head()

relation         left    right
0   hyponym        bears    carry
1  hypernym         seat  portion
2   synonym        frame   adjust
3   synonym        grass   spring
4   synonym  anniversary   yearly

Some pairs have multiple relations, which should be impossible. Sample to avoid this:

In [5]:
df = df.groupby(['left', 'right']).apply(lambda x: x.sample(1))

In [6]:
df = df.drop(['left', 'right'], axis=1).reset_index(['left', 'right']).reset_index(drop=True)

In [7]:
df = df[(df['left'].isna()==False) & (df['right'].isna()==False)].copy()

In [8]:
df.shape[0]

28355

## Random train–test split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    df[['left', 'right']].values, df['relation'].values, 
    random_state=42, train_size=0.6, test_size=None)

In [10]:
train = list(zip(X_train, y_train))

Make sure there are no pairs in both train and test (the original dataset has a handful of repeated pairs that can lead to violations of this expectation of disjointness):

In [11]:
test_set = {tuple(x) for x in X_test}

In [12]:
X_train = [[list(x), y] for x, y in train if tuple(x) not in test_set]

In [13]:
test = list(zip(X_test, y_test))

In [14]:
X_test = [[list(x), y] for x, y in test]

In [15]:
len(X_train)

17013

In [16]:
len(X_test)

11342

In [17]:
len({tuple(x) for x in list(zip(*X_train))[0]})

17013

## Fully disjoint train-test split

In [18]:
vocab = sorted(set(df[['left', 'right']].values.ravel()))

In [19]:
train_size = int(len(vocab) * 0.6)

In [20]:
train_vocab = sorted(random.sample(vocab, train_size))

In [21]:
test_vocab = sorted(set(vocab) - set(train_vocab))

In [22]:
def create_disjoint_df_split(vocab):
    return df[(df['left'].isin(vocab)) & (df['right'].isin(vocab))]


disjoint_train = create_disjoint_df_split(train_vocab)

disjoint_test = create_disjoint_df_split(test_vocab)

In [23]:
disjoint_total = disjoint_train.shape[0] + disjoint_test.shape[0]

In [24]:
disjoint_total

14864

In [25]:
disjoint_total / df.shape[0]

0.5242108975489331

In [26]:
disjoint_train.shape[0] / disjoint_total

0.7142088266953713

In [27]:
d_train = disjoint_train['relation'].value_counts()

d_train / d_train.sum()

synonym     0.528448
hypernym    0.376130
hyponym     0.059062
antonym     0.036360
Name: relation, dtype: float64

In [28]:
d_test = disjoint_test['relation'].value_counts()

d_test / d_test.sum()

synonym     0.524718
hypernym    0.375235
hyponym     0.064736
antonym     0.035311
Name: relation, dtype: float64

In [29]:
def create_disjoint_split(disjoint):
    return [[[x['left'], x['right']], x['relation']] for _, x in disjoint.iterrows()]


X_disjoint_train = create_disjoint_split(disjoint_train)

X_disjoint_test = create_disjoint_split(disjoint_test)

In [30]:
dtr, _ = zip(*X_disjoint_train)
dtr = set(map(tuple, dtr))

dt, _ = zip(*X_disjoint_test)
dt = set(map(tuple, dt))

dtr & dt

set()

## Disjoint and balanced

In [48]:
def balance_split(split):   
    data = []
    seen_left = set()    
    seen_right = set()
    random.shuffle(split)
    for (left, right), label in split:
        if (left, label) not in seen_left and (right, label) not in seen_right:
            data.append([[left, right], label])
            seen_left.add((left, label))
            seen_right.add((right, label))
    return data

In [49]:
X_balanced_train = balance_split(X_disjoint_train)

In [50]:
X_balanced_test = balance_split(X_disjoint_test)

In [51]:
len(X_balanced_train)

2949

In [52]:
len(X_balanced_test)

1575

In [53]:
pd.DataFrame(X_balanced_train)[1].value_counts()

synonym     1530
hypernym     960
hyponym      236
antonym      223
Name: 1, dtype: int64

In [54]:
pd.DataFrame(X_balanced_test)[1].value_counts()

synonym     831
hypernym    511
hyponym     118
antonym     115
Name: 1, dtype: int64

In [63]:
vocab = {w for ex, label in X_train+X_test for w in ex}

In [64]:
len(vocab)

6560

## Output in JSON

In [66]:
data = {
    'edge_disjoint': {
        'train': X_train, 
        'dev': X_test
    },
    'word_disjoint': {
        'train': X_disjoint_train,
        'dev': X_disjoint_test
    },
    'word_disjoint_balanced': {
        'train': X_balanced_train,
        'dev': X_balanced_test        
    },
    'vocab': sorted(vocab)
}

In [67]:
len(data['edge_disjoint']['train'])

17013

In [68]:
len(data['word_disjoint']['train'])

10616

In [69]:
len(data['edge_disjoint']['dev'])

11342

In [70]:
len(data['word_disjoint']['dev'])

4248

In [71]:
len(data['word_disjoint_balanced']['train'])

2949

In [72]:
len(data['word_disjoint_balanced']['dev'])

1575

In [73]:
with open('nli_wordentail_bakeoff_data.json', 'wt') as f:
    json.dump(data, f, indent=4, sort_keys=True)